In [ ]:
import torch
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.nn as nn

import pandas as pd
import numpy as np

class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self, phase='train'):
   # 데이터 읽기
   f = pd.read_csv('wine.csv')
   data = f.to_numpy()
   if phase=='train':
     data=data[:1200,:]
   if phase=='val':
     data=data[1201:,:]

   # 전처리
   self.x_data = np.array(data[:,0:11], dtype=np.float)
   self.x_data = ( self.x_data -  self.x_data.min(axis=0)) / ( self.x_data.max(axis=0) -  self.x_data.min(axis=0))

   self.y_data = data[:, 11:12]
   self.y_data[self.y_data=='bad']=0
   self.y_data[self.y_data=='good']=1
   self.y_data=np.array(self.y_data, dtype=np.long)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx): 
    x = torch.Tensor(self.x_data[idx])
    y = torch.LongTensor(self.y_data[idx])
    return x, y

batch_=10
dataloader ={ 'train' :  DataLoader(CustomDataset(phase='train'), batch_size=batch_, shuffle=True),
               'val' :  DataLoader(CustomDataset(phase='val'), batch_size=batch_, shuffle=False)
 }


## Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11, 50)    
        self.fc2 = nn.Linear(50, 20)   
        self.fc3 = nn.Linear(20, 2)
        
    def forward(self, x):
        # flatten image input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

model = Net() # 모델 구축
loss_fn =  torch.nn.CrossEntropyLoss()  # 손실함수 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 매개변수 갱신

nb_epochs = 400

for epoch in range(0, nb_epochs):
  avg_cost = 0
  sample_size = 0
  avg_cost_ = 0
  sample_size_ = 0

  # Train
  for batch_idx, samples in enumerate(dataloader['train']):
    x_train, y_train = samples

    pred = model(x_train)
    loss = loss_fn(pred, y_train[:,0])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    avg_cost+=loss
    sample_size+=1

  # Val
  for batch_idx, samples in enumerate(dataloader['val']):
    x_train, y_train = samples
    pred = model(x_train)
    loss = loss_fn(pred, y_train[:,0])
    avg_cost_+=loss
    sample_size_+=1
  
  print("epoch :" , epoch, "loss_train", avg_cost/sample_size, "loss_val", avg_cost_/sample_size_ )